## AIGCDetectBenchmark


## Install requirement

In [1]:
!git clone https://github.com/vohoaidanh/ADOF.git

Cloning into 'ADOF'...
remote: Enumerating objects: 870, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 870 (delta 43), reused 49 (delta 41), pack-reused 817 (from 1)
Receiving objects: 100% (870/870), 28.78 MiB | 15.32 MiB/s, done.
Resolving deltas: 100% (572/572), done.


In [2]:
%cd ADOF

/content/ADOF


In [3]:
!pip install -r requirements.txt
!pip install tensorboardX -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.6 MB/s eta 0:00:00


In [4]:
# Without google colab
COLAB = True
if not COLAB:
  !pip install gdown -q
  !apt-get install -y unzip -q
  !apt-get install -y zip -q
  !pip install tensorboardX -q
  !pip install regex -q
  !pip install imageio -q
  !pip install opencv-python -q
  !apt-get install -y libgl1-mesa-glx -q
  !pip install scikit-learn -q
  !pip install scikit-image -q
  !pip install ftfy -q
  !pip install natsort -q
  !pip install blobfile -q

## Download Data

In [ ]:
#Download Train, val set
# https://drive.google.com/file/d/1JW-4upuC3Tqq9awz8gnRWEf61NncC8JT/view?usp=drive_link    (Full train, val set)
# https://drive.google.com/file/d/1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z/view?usp=drive_link     (Small tran, val set)
import gdown
DATASET_PATH = 'datasets'
TRAIN_SET_PATH = 'ForenSynths_train'
file_id = '1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z' #Progan train/val 4 class [car, cat, chair, horse]
filename_train = 'progan_train.zip'  # Desired file name and extension
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, filename_train, quiet=False)
#Unzip
!mkdir -p $DATASET_PATH/$TRAIN_SET_PATH
!mkdir -p $DATASET_PATH/$TRAIN_SET_PATH/test
!unzip -q $filename_train -d $DATASET_PATH/$TRAIN_SET_PATH

Downloading...
From (original): https://drive.google.com/uc?id=1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z
From (redirected): https://drive.google.com/uc?id=1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z&confirm=t&uuid=6c860d9b-9fab-4829-b947-97540cd5c9ef
To: /content/ADOF/progan_train.zip
100%|██████████| 2.64G/2.64G [00:26<00:00, 98.2MB/s]


In [5]:
#Download small Test set (this is a small testset just use for reference during training)

import gdown
file_id = '1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4' #test_set
filename_test = 'test_set.zip'  # Desired file name and extension
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, filename_test, quiet=False)
#Unzip
!unzip -q $filename_test -d $DATASET_PATH/$TRAIN_SET_PATH/test

Downloading...
From (original): https://drive.google.com/uc?id=1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4
From (redirected): https://drive.google.com/uc?id=1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4&confirm=t&uuid=8c039f27-2a43-45b3-b7e4-a856f0c5f5d6
To: /content/ADOF/test_set.zip
100%|██████████| 1.68G/1.68G [00:20<00:00, 83.5MB/s]


UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files as text
  -U  use escapes for all non-ASCII Unicode  -UU ignore any Unicode fields
  -C  mat

In [9]:
!unzip -q $filename_test -d $DATASET_PATH/$TRAIN_SET_PATH/test

## Train

In [ ]:
#experiment-01-no-filter
!find $DATASET_PATH -type d -name "*ipynb*" -exec rm -r {} +
!python train.py \
--name adof-progan-4class- \
--dataroot $DATASET_PATH/$TRAIN_SET_PATH \
--num_thread 2 \
--classes car,cat,chair,horse --batch_size 32 --delr_freq 5 --lr 0.0002 --niter 30

----------------- Options ---------------
                     arch: res50                         
               batch_size: 32                            	[default: 64]
                    beta1: 0.9                           
                blur_prob: 0                             
                 blur_sig: 0.5                           
          checkpoints_dir: ./checkpoints                 
                class_bal: False                         
                  classes: car,cat,chair,horse           	[default: ]
           continue_train: False                         
                 cropSize: 224                           
                 data_aug: False                         
                 dataroot: datasets/ForenSynths_train    	[default: ./dataset/]
                delr_freq: 5                             	[default: 20]
          earlystop_epoch: 15                            
                    epoch: latest                        
              epoch_count:

## Evaluation

In [ ]:
!chmod +x download_dataset.sh
!sed -i 's/\r$//' download_dataset.sh
!./download_dataset.sh

In [13]:
# You should update the test set directory in the "test.py" file.
!find $DATASET_PATH -type d -name "*ipynb*" -exec rm -r {} +
!python test.py \
--model_path ./weights/ADOF_model_epoch_9.pth  \
--num_thread 2 \
--batch_size 32

Model_path ./weights/ADOF_model_epoch_9.pth
           ForenSynths
2025_09_10_16_24_58
(0 stylegan2   ) acc: 99.4; ap: 100.0; r_acc: 1.0; f_acc: 1.0
(1 progan      ) acc: 99.8; ap: 100.0; r_acc: 1.0; f_acc: 1.0
(2 gaugan      ) acc: 71.0; ap: 81.4; r_acc: 0.5; f_acc: 0.9
(3 cyclegan    ) acc: 85.6; ap: 94.1; r_acc: 0.9; f_acc: 0.8
(4 stargan     ) acc: 96.8; ap: 100.0; r_acc: 0.9; f_acc: 1.0
(5 deepfake    ) acc: 85.0; ap: 92.8; r_acc: 0.9; f_acc: 0.8
(6 stylegan    ) acc: 99.8; ap: 100.0; r_acc: 1.0; f_acc: 1.0
(7 biggan      ) acc: 80.0; ap: 91.5; r_acc: 0.6; f_acc: 1.0
(8 Mean      ) acc: 89.7; ap: 95.0
*************************


In [16]:
import gdown

# Link folder dạng "uc?id=FOLDER_ID&export=download"
folder_id = "1HdoWolRAia6QvUDr02lPX684XCSqEd71"
url = f"https://drive.google.com/uc?id={folder_id}&export=download"
output = "Trainable.zip"

gdown.download(url, output, quiet=False)


FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1HdoWolRAia6QvUDr02lPX684XCSqEd71&export=download

but Gdown can't. Please check connections and permissions.